In [1]:
import os
import json
import yaml
import openai
from rich import print
from langchain.output_parsers import PydanticOutputParser
from dotenv import load_dotenv, find_dotenv
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import pandas as pd
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

load_dotenv(find_dotenv())

openai_api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
with open('./questions.json') as f:
    questions_list = json.load(f)['questions']

In [3]:
from typing import List, Optional
from pydantic import BaseModel, Field

class D(BaseModel):
    val: int
    unit: str

class Ex(BaseModel):
    name: str
    type: str
    sets: str = Field(default = None)
    reps: str = Field(default = None)
    dur: Optional[D] = Field(default = None)

class Day(BaseModel):
    num: int
    focus: str
    exs: Optional[List[Ex]]

class Wk(BaseModel):
    wk_range: str
    days: List[Day]

class N(BaseModel):
    content: str

class WorkoutPlan(BaseModel):
    wks: List[Wk]
    notes: List[N]

In [20]:
system_message = SystemMessagePromptTemplate.from_template(
    """Your task is to act as a personal fitness trainer. 
    You will be provided with information about your CLIENT's current fitness level, their goals, and other preferences they have. 
    You have to prepare a detailed workout plan for the CLIENT, including everything essential for physical fitness.
    Ensure that you strictly adhere to the number of days and weeks the CLIENT is willing to do their workout."""  # noqa: E501
)

In [21]:
question_message = HumanMessagePromptTemplate.from_template(
    f'The list of questions that have been asked to the CLIENT (separated by a ;) are: \n{";".join(i["text"] for i in questions_list)}'
)

print(question_message)

prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='The list of questions
that have been asked to the CLIENT (separated by a ;) are: \nWhat is your age range?;What is your gender?;What is 
your primary goal for strength training?;How would you describe your current fitness level?;What is your current 
height (in cms)?;What is your current weight (in kgs)?;Have you ever followed a structured strength training 
program before?;Do you have any previous injuries or medical conditions that may affect your training?;How many 
days per week can you dedicate to strength training?;How many weeks do you plan to workout?;Do you have access to a
gym or will you be working out at home?;Which of these muscle groups do you consider your weakest or want to 
improve the most?;How would you rate your experience with strength training exercises?;What type of strength 
training program do you prefer?;Do you prefer to focus on bodyweight exercises, free weights, or a mix of both?;How
important is increasing your cardiovascular fitness to you?', template_format='f-string', validate_template=True) 
additional_kwargs={}

In [22]:
answer_message = HumanMessagePromptTemplate.from_template(
    "The answers provided by the CLIENT (separated by a ;) are: \n18-24;Male;Lose body fat and tone muscles;Beginner (little or no experience);180.0;79.0;No;No;Upper body (chest, shoulders, arms);Beginner (little to no experience);4 days;6 weeks;Gym access;Push, pull, legs (PPL) split;Free weights;Important"
)

print(answer_message)

prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='The answers provided 
by the CLIENT (separated by a ;) are: \n18-24;Male;Lose body fat and tone muscles;Beginner (little or no 
experience);180.0;79.0;No;No;Upper body (chest, shoulders, arms);Beginner (little to no experience);4 days;6 
weeks;Gym access;Push, pull, legs (PPL) split;Free weights;Important', template_format='f-string', 
validate_template=True) additional_kwargs={}

In [23]:
answers = [
    "Under 18",
    "Male",
    "Increase muscle mass",
    "Beginner (little or no experience)",
    50,
    50,
    "Yes",
    "No",
    "Core (abs, lower back)",
    "Beginner (little to no experience)",
    "2 days",
    "4 weeks",
    "Gym access",
    "Full body workouts",
    "Bodyweight exercises",
    "Not important",
]

In [24]:
qa_message = HumanMessagePromptTemplate.from_template(
    template="""{question}
    CLIENT: {answer}
    """
)

In [25]:
schema = """
WorkoutPlan:
  wks:
    - wk_range: Week or range of weeks in the plan (e.g., Week 1-2).
    days:
      - num: Day number within the workout plan.
      - focus: Primary muscle group or activity targeted on this day.
      exs:
        - name: Name of the exercise.
        - type: Category of the exercise (warmup, cardio, stretching, or strength).
        - sets: Number of sets to perform.
        - reps: Number of repetitions per set.
        dur:
          val: Duration value of the exercise.
          unit: Time unit for the duration (e.g., seconds, minutes).
  notes:
    - content: Additional information, tips, or instructions related to the workout plan as individual items.
"""

In [26]:
example_response = """
WorkoutPlan:
  wks:
    - wk_range: Week 1-2
      days:
        - num: 1
          focus: Body Part A
          exs:
            - name: Exercise A1
              type: Type A
              sets: 3
              reps: 10
            - name: Exercise A2
              type: Type C
              sets: 3
              reps: 12
            - name: Exercise A3
              type: Type B
              dur:
                val: 30
                unit: seconds
        - num: 2
          focus: Body Part B
          exs:
            - name: Exercise B1
              type: Type A
              sets: 4
              reps: 8
            - name: Exercise B2
              type: Type A
              sets: 3
              reps: 10
            - name: Exercise B3
              type: Type C
              dur:
                val: 1
                unit: minute
  notes:
    - content: Note 1
    - content: Note 2
"""

In [27]:
format_message = HumanMessagePromptTemplate.from_template(
    template="""With the given details, proceed to create a detailed weekly plan that can be followed for 2 months for the CLIENT, making sure to incorporate their inputs properly. 
    Ensure that you include cardio and rest time inside the plan as part of the `exercise` object.  If a day is classified as "rest day", skip it from your output. 
    Try to keep the notes section as generic as possible as most CLIENTS will not read it."""
)

In [28]:
schema_message = HumanMessagePromptTemplate.from_template(
    template="""
    The output should be formatted in a way that conforms to the given schema below. As an example, a valid response is: 
    {example_response}
    
    Here is the output schema:
    ```
    {format_instructions}
    ```
    """
)

print(format_message, schema_message)

prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='With the given 
details, proceed to create a detailed weekly plan that can be followed for 2 months for the CLIENT, making sure to 
incorporate their inputs properly. \n    Ensure that you include cardio and rest time inside the plan as part of 
the `exercise` object.  If a day is classified as "rest day", skip it from your output. \n    Try to keep the notes
section as generic as possible as most CLIENTS will not read it.', template_format='f-string', 
validate_template=True) additional_kwargs={} prompt=PromptTemplate(input_variables=['example_response', 
'format_instructions'], output_parser=None, partial_variables={}, template='\n    The output should be formatted in
a way that conforms to the given schema below. As an example, a valid response is: \n    {example_response}\n    \n
Here is the output schema:\n    ```\n    {format_instructions}\n    ```\n    ', template_format='f-string', 
validate_template=True) additional_kwargs={}

In [47]:
qa_messages = [qa_message.format(question=q["text"], answer=answers[i]) for i, q in enumerate(questions_list)]
qa_messages

[HumanMessage(content='What is your age range?\n    CLIENT: Under 18\n    ', additional_kwargs={}, example=False),
 HumanMessage(content='What is your gender?\n    CLIENT: Male\n    ', additional_kwargs={}, example=False),
 HumanMessage(content='What is your primary goal for strength training?\n    CLIENT: Increase muscle mass\n    ', additional_kwargs={}, example=False),
 HumanMessage(content='How would you describe your current fitness level?\n    CLIENT: Beginner (little or no experience)\n    ', additional_kwargs={}, example=False),
 HumanMessage(content='What is your current height (in cms)?\n    CLIENT: 50\n    ', additional_kwargs={}, example=False),
 HumanMessage(content='What is your current weight (in kgs)?\n    CLIENT: 50\n    ', additional_kwargs={}, example=False),
 HumanMessage(content='Have you ever followed a structured strength training program before?\n    CLIENT: Yes\n    ', additional_kwargs={}, example=False),
 HumanMessage(content='Do you have any previous injurie

In [37]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        format_message,
        schema_message,
    ]
)

In [48]:
messages = chat_prompt.format_prompt(
    example_response=example_response,
    format_instructions=schema,
).to_messages()

for x in reversed(qa_messages):
    messages.insert(1, x) 
print(messages)

[
    SystemMessage(content="Your task is to act as a personal fitness trainer. \n    You will be provided with 
information about your CLIENT's current fitness level, their goals, and other preferences they have. \n    You have
to prepare a detailed workout plan for the CLIENT, including everything essential for physical fitness.\n    Ensure
that you strictly adhere to the number of days and weeks the CLIENT is willing to do their workout.", 
additional_kwargs={}),
    HumanMessage(content='What is your age range?\n    CLIENT: Under 18\n    ', additional_kwargs={}, 
example=False),
    HumanMessage(content='What is your gender?\n    CLIENT: Male\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='What is your primary goal for strength training?\n    CLIENT: Increase muscle mass\n    
', additional_kwargs={}, example=False),
    HumanMessage(content='How would you describe your current fitness level?\n    CLIENT: Beginner (little or no 
experience)\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='What is your current height (in cms)?\n    CLIENT: 50\n    ', additional_kwargs={}, 
example=False),
    HumanMessage(content='What is your current weight (in kgs)?\n    CLIENT: 50\n    ', additional_kwargs={}, 
example=False),
    HumanMessage(content='Have you ever followed a structured strength training program before?\n    CLIENT: Yes\n 
', additional_kwargs={}, example=False),
    HumanMessage(content='Do you have any previous injuries or medical conditions that may affect your training?\n 
CLIENT: No\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='How many days per week can you dedicate to strength training?\n    CLIENT: Core (abs, 
lower back)\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='How many weeks do you plan to workout?\n    CLIENT: Beginner (little to no experience)\n 
', additional_kwargs={}, example=False),
    HumanMessage(content='Do you have access to a gym or will you be working out at home?\n    CLIENT: 2 days\n    
', additional_kwargs={}, example=False),
    HumanMessage(content='Which of these muscle groups do you consider your weakest or want to improve the most?\n 
CLIENT: 4 weeks\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='How would you rate your experience with strength training exercises?\n    CLIENT: Gym 
access\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='What type of strength training program do you prefer?\n    CLIENT: Full body workouts\n  
', additional_kwargs={}, example=False),
    HumanMessage(content='Do you prefer to focus on bodyweight exercises, free weights, or a mix of both?\n    
CLIENT: Bodyweight exercises\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='How important is increasing your cardiovascular fitness to you?\n    CLIENT: Not 
important\n    ', additional_kwargs={}, example=False),
    HumanMessage(content='With the given details, proceed to create a detailed weekly plan that can be followed for
2 months for the CLIENT, making sure to incorporate their inputs properly. \n    Ensure that you include cardio and
rest time inside the plan as part of the `exercise` object.  If a day is classified as "rest day", skip it from 
your output. \n    Try to keep the notes section as generic as possible as most CLIENTS will not read it.', 
additional_kwargs={}, example=False),
    HumanMessage(content='\n    The output should be formatted in a way that conforms to the given schema below. As
an example, a valid response is: \n    \nWorkoutPlan:\n  wks:\n    - wk_range: Week 1-2\n      days:\n        - 
num: 1\n          focus: Body Part A\n          exs:\n            - name: Exercise A1\n              type: Type A\n
sets: 3\n              reps: 10\n            - name: Exercise A2\n              type: Type C\n              sets: 
3\n              reps: 12\n            - name: Exercise A3\n              type: Type B\n          

In [49]:
chat = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.2,
    openai_api_key=openai_api_key,
    verbose=True,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
chat_turbo = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.2,
    openai_api_key=openai_api_key,
    verbose=True,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [51]:
response = chat(messages)

WorkoutPlan:
  wks:
    - wk_range: Week 1-2
      days:
        - num:

KeyboardInterrupt: 

In [66]:
print(response.content)

WorkoutPlan:
  wks:
    - wk_range: Week 1-2
      days:
        - num: 1
          focus: Push (Chest, Shoulders, Triceps)
          exs:
            - name: Bench Press
              type: Strength
              sets: 3
              reps: 10
            - name: Overhead Press
              type: Strength
              sets: 3
              reps: 10
            - name: Tricep Dips
              type: Strength
              sets: 3
              reps: 12
            - name: Treadmill
              type: Cardio
              dur:
                val: 20
                unit: minutes
        - num: 2
          focus: Pull (Back, Biceps)
          exs:
            - name: Deadlift
              type: Strength
              sets: 3
              reps: 8
            - name: Pull-ups
              type: Strength
              sets: 3
              reps: 10
            - name: Barbell Rows
              type: Strength
              sets: 3
              reps: 10
            - name: Stationary Bike
              type: Cardio
              dur:
                val: 20
                unit: minutes
        - num: 3
          focus: Legs (Quads, Hamstrings, Calves)
          exs:
            - name: Squats
              type: Strength
              sets: 3
              reps: 10
            - name: Leg Press
              type: Strength
              sets: 3
              reps: 10
            - name: Calf Raises
              type: Strength
              sets: 3
              reps: 15
            - name: Jump Rope
              type: Cardio
              dur:
                val: 10
                unit: minutes
        - num: 4
          focus: Rest Day
    - wk_range: Week 3-4
      days:
        - num: 1
          focus: Push (Chest, Shoulders, Triceps)
          exs:
            - name: Incline Bench Press
              type: Strength
              sets: 3
              reps: 10
            - name: Dumbbell Shoulder Press
              type: Strength
              sets: 3
              reps: 10
            - name: Skull Crushers
              type: Strength
              sets: 3
              reps: 12
            - name: Treadmill
              type: Cardio
              dur:
                val: 25
                unit: minutes
        - num: 2
          focus: Pull (Back, Biceps)
          exs:
            - name: Deadlift
              type: Strength
              sets: 3
              reps: 8
            - name: Chin-ups
              type: Strength
              sets: 3
              reps: 10
            - name: Seated Cable Rows
              type: Strength
              sets: 3
              reps: 10
            - name: Stationary Bike
              type: Cardio
              dur:
                val: 25
                unit: minutes
        - num: 3
          focus: Legs (Quads, Hamstrings, Calves)
          exs:
            - name: Front Squats
              type: Strength
              sets: 3
              reps: 10
            - name: Leg Curl
              type: Strength
              sets: 3
              reps: 10
            - name: Seated Calf Raises
              type: Strength
              sets: 3
              reps: 15
            - name: Jump Rope
              type: Cardio
              dur:
                val: 15
                unit: minutes
        - num: 4
          focus: Rest Day
    - wk_range: Week 5-6
      days:
        - num: 1
          focus: Push (Chest, Shoulders, Triceps)
          exs:
            - name: Decline Bench Press
              type: Strength
              sets: 3
              reps: 10
            - name: Lateral Raises
              type: Strength
              sets: 3
              reps: 10
            - name: Close Grip Bench Press
              type: Strength
              sets: 3
              reps: 12
            - name: Treadmill
              type: Cardio
              dur:
                val: 30
                unit: minutes
        - num: 2
          focus: Pull (Back, Bi

In [74]:
response_json = yaml.safe_load(response.content)

with open('response.json', 'w') as f:
    json.dump(response_json, f)

plan = WorkoutPlan(**response_json['WorkoutPlan'])

In [75]:
print(plan)

wks=[Wk(wk_range='Week 1-2', days=[Day(num=1, focus='Push (Chest, Shoulders, Triceps)', exs=[Ex(name='Bench Press',
type='Strength', sets='3', reps='10', dur=None), Ex(name='Overhead Press', type='Strength', sets='3', reps='10', 
dur=None), Ex(name='Tricep Dips', type='Strength', sets='3', reps='12', dur=None), Ex(name='Treadmill', 
type='Cardio', sets=None, reps=None, dur=D(val=20, unit='minutes'))]), Day(num=2, focus='Pull (Back, Biceps)', 
exs=[Ex(name='Deadlift', type='Strength', sets='3', reps='8', dur=None), Ex(name='Pull-ups', type='Strength', 
sets='3', reps='10', dur=None), Ex(name='Barbell Rows', type='Strength', sets='3', reps='10', dur=None), 
Ex(name='Stationary Bike', type='Cardio', sets=None, reps=None, dur=D(val=20, unit='minutes'))]), Day(num=3, 
focus='Legs (Quads, Hamstrings, Calves)', exs=[Ex(name='Squats', type='Strength', sets='3', reps='10', dur=None), 
Ex(name='Leg Press', type='Strength', sets='3', reps='10', dur=None), Ex(name='Calf Raises', type='Strength', 
sets='3', reps='15', dur=None), Ex(name='Jump Rope', type='Cardio', sets=None, reps=None, dur=D(val=10, 
unit='minutes'))]), Day(num=4, focus='Rest Day', exs=None)]), Wk(wk_range='Week 3-4', days=[Day(num=1, focus='Push 
(Chest, Shoulders, Triceps)', exs=[Ex(name='Incline Bench Press', type='Strength', sets='3', reps='10', dur=None), 
Ex(name='Dumbbell Shoulder Press', type='Strength', sets='3', reps='10', dur=None), Ex(name='Skull Crushers', 
type='Strength', sets='3', reps='12', dur=None), Ex(name='Treadmill', type='Cardio', sets=None, reps=None, 
dur=D(val=25, unit='minutes'))]), Day(num=2, focus='Pull (Back, Biceps)', exs=[Ex(name='Deadlift', type='Strength',
sets='3', reps='8', dur=None), Ex(name='Chin-ups', type='Strength', sets='3', reps='10', dur=None), Ex(name='Seated
Cable Rows', type='Strength', sets='3', reps='10', dur=None), Ex(name='Stationary Bike', type='Cardio', sets=None, 
reps=None, dur=D(val=25, unit='minutes'))]), Day(num=3, focus='Legs (Quads, Hamstrings, Calves)', 
exs=[Ex(name='Front Squats', type='Strength', sets='3', reps='10', dur=None), Ex(name='Leg Curl', type='Strength', 
sets='3', reps='10', dur=None), Ex(name='Seated Calf Raises', type='Strength', sets='3', reps='15', dur=None), 
Ex(name='Jump Rope', type='Cardio', sets=None, reps=None, dur=D(val=15, unit='minutes'))]), Day(num=4, focus='Rest 
Day', exs=None)]), Wk(wk_range='Week 5-6', days=[Day(num=1, focus='Push (Chest, Shoulders, Triceps)', 
exs=[Ex(name='Decline Bench Press', type='Strength', sets='3', reps='10', dur=None), Ex(name='Lateral Raises', 
type='Strength', sets='3', reps='10', dur=None), Ex(name='Close Grip Bench Press', type='Strength', sets='3', 
reps='12', dur=None), Ex(name='Treadmill', type='Cardio', sets=None, reps=None, dur=D(val=30, unit='minutes'))]), 
Day(num=2, focus='Pull (Back, Biceps)', exs=[Ex(name='Deadlift', type='Strength', sets='3', reps='8', dur=None), 
Ex(name='Hammer Curls', type='Strength', sets='3', reps='10', dur=None), Ex(name='Lat Pulldown', type='Strength', 
sets='3', reps='10', dur=None), Ex(name='Stationary Bike', type='Cardio', sets=None, reps=None, dur=D(val=30, 
unit='minutes'))]), Day(num=3, focus='Legs (Quads, Hamstrings, Calves)', exs=[Ex(name='Lunges', type='Strength', 
sets='3', reps='10', dur=None), Ex(name='Leg Extension', type='Strength', sets='3', reps='10', dur=None), 
Ex(name='Standing Calf Raises', type='Strength', sets='3', reps='15', dur=None), Ex(name='Jump Rope', 
type='Cardio', sets=None, reps=None, dur=D(val=20, unit='minutes'))]), Day(num=4, focus='Rest Day', exs=None)])] 
notes=[N(content='Warm up for 5-10 minutes before each workout.'), N(content='Stretch for 5-10 minutes after each 
workout.'), N(content='Adjust weights and intensity as needed to maintain proper form and progress.'), 
N(content='Stay hydrated and maintain a balanced diet to support your training goals.')]

In [93]:
def convert_to_dataframe(plan):
    weeks, notes = plan.wks, plan.notes

    workout_data = []
    for week in weeks:
        week_range, days = week.wk_range, week.days
        for day in days:
            if day.exs:
                for exercise in day.exs:
                    row = {
                        "Week Range": week_range,
                        "Day Number": day.num,
                        "Focus": day.focus,
                        "Exercise Name": exercise.name,
                        "Exercise Type": exercise.type,
                        "Sets": exercise.sets,
                        "Reps": exercise.reps,
                        "Duration": f"{exercise.dur.val} {exercise.dur.unit}"
                        if exercise.dur
                        else None,
                    }
                    workout_data.append(row)
            else:
                row = {
                    "Week Range": week_range,
                    "Day Number": day.num,
                    "Focus": day.focus,
                    "Exercise Name": None,
                    "Exercise Type": None,
                    "Sets": None,
                    "Reps": None,
                    "Duration": None,
                }
                workout_data.append(row)

    note_data = []
    for note in notes:
        row = {"note": note.content}
        note_data.append(row)

    return workout_data, note_data

In [94]:
pd.DataFrame(convert_to_dataframe(plan)[0])

,Week Range,Day Number,Focus,Exercise Name,Exercise Type,Sets,Reps,Duration
0,Week 1-2,1,"Push (Chest, Shoulders, Triceps)",Bench Press,Strength,3,10,None
1,Week 1-2,1,"Push (Chest, Shoulders, Triceps)",Overhead Press,Strength,3,10,None
2,Week 1-2,1,"Push (Chest, Shoulders, Triceps)",Tricep Dips,Strength,3,12,None
3,Week 1-2,1,"Push (Chest, Shoulders, Triceps)",Treadmill,Cardio,None,None,20 minutes
4,Week 1-2,2,"Pull (Back, Biceps)",Deadlift,Strength,3,8,None
5,Week 1-2,2,"Pull (Back, Biceps)",Pull-ups,Strength,3,10,None
6,Week 1-2,2,"Pull (Back, Biceps)",Barbell Rows,Strength,3,10,None
7,Week 1-2,2,"Pull (Back, Biceps)",Stationary Bike,Cardio,None,None,20 minutes
8,Week 1-2,3,"Legs (Quads, Hamstrings, Calves)",Squats,Strength,3,10,None
9,Week 1-2,3,"Legs (Quads, Hamstrings, Calves)",Leg Press,Strength,3,10,None


In [30]:
weeks = WorkoutPlan(
    weeks=[
        Week(
            week_range="Week 1-2",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Upper Body (Chest, Shoulders, Triceps)",
                    exercises=[
                        Exercise(
                            name="Push-ups",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Dumbbell Shoulder Press",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Tricep Dips",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bench Dips",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=20, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=20, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
        Week(
            week_range="Week 3-4",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Lower Body (Legs, Glutes)",
                    exercises=[
                        Exercise(
                            name="Squats",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Lunges",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Leg Press",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Calf Raises",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=25, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=25, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
        Week(
            week_range="Week 5-6",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Upper Body (Back, Biceps)",
                    exercises=[
                        Exercise(
                            name="Pull-ups",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                        Exercise(
                            name="Dumbbell Rows",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicep Curls",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Hammer Curls",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=30, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=30, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
        Week(
            week_range="Week 7-8",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Full Body",
                    exercises=[
                        Exercise(
                            name="Deadlifts",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                        Exercise(
                            name="Squats",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bench Press",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                        Exercise(
                            name="Pull-ups",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=35, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=35, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
    ],
    notes=[
        Note(
            content="Remember to stretch before and after each workout to prevent injury."
        ),
        Note(
            content="Make sure to stay hydrated throughout the day and during workouts."
        ),
        Note(
            content="If you experience any pain or discomfort during exercises, stop immediately and consult a doctor."
        ),
    ],
)

In [177]:
pd.DataFrame(convert_to_dataframe(WorkoutPlan(**response_json['WorkoutPlan']))[0])

AttributeError: 'WorkoutPlan' object has no attribute 'weeks'

In [33]:
unstructured_response = response.content
# print(unstructured_response)

unstructured_format_message = HumanMessagePromptTemplate.from_template(
    template="Please structure the given block of text according to the format instructions provided.\n===\n{text_block}\n===FORMAT INSTRUCTIONS: {format_instructions}"
)

chat_prompt_format = ChatPromptTemplate.from_messages(
    [unstructured_format_message]
)

messages_format = chat_prompt_format.format_prompt(text_block=unstructured_response, format_instructions=parser.get_format_instructions()).to_messages()

print(messages_format)

[
    HumanMessage(content='Please structure the given block of text according to the format instructions 
provided.\n===\nBased on the information provided, I have created a detailed 6-week workout plan for the CLIENT. 
The plan follows a Push, Pull, Legs (PPL) split, focusing on free weights and incorporating cardiovascular fitness.
The CLIENT will be working out 4 days a week. Here\'s the weekly plan:\n\nDay 1: Push (Chest, Shoulders, 
Triceps)\n1. Warm-up: 5 minutes of light cardio (jogging, jumping jacks, etc.)\n2. Bench Press: 3 sets of 10-12 
reps\n3. Overhead Press: 3 sets of 10-12 reps\n4. Incline Dumbbell Press: 3 sets of 10-12 reps\n5. Lateral Raises: 
3 sets of 12-15 reps\n6. Tricep Dips: 3 sets of 10-12 reps\n7. Cool down: Stretching exercises for 5 minutes\n\nDay
2: Pull (Back, Biceps)\n1. Warm-up: 5 minutes of light cardio (jogging, jumping jacks, etc.)\n2. Pull-ups: 3 sets 
of 8-10 reps\n3. Bent Over Rows: 3 sets of 10-12 reps\n4. Lat Pulldowns: 3 sets of 10-12 reps\n5. Seated Rows: 3 
sets of 10-12 reps\n6. Bicep Curls: 3 sets of 10-12 reps\n7. Cool down: Stretching exercises for 5 minutes\n\nDay 
3: Rest\n\nDay 4: Legs (Quads, Hamstrings, Calves)\n1. Warm-up: 5 minutes of light cardio (jogging, jumping jacks, 
etc.)\n2. Squats: 3 sets of 10-12 reps\n3. Leg Press: 3 sets of 10-12 reps\n4. Lunges: 3 sets of 10-12 reps (each 
leg)\n5. Leg Curls: 3 sets of 10-12 reps\n6. Calf Raises: 3 sets of 15-20 reps\n7. Cool down: Stretching exercises 
for 5 minutes\n\nDay 5: Cardio\n1. Warm-up: 5 minutes of light cardio (jogging, jumping jacks, etc.)\n2. Choose one
of the following: 30 minutes of moderate-intensity running, cycling, swimming, or rowing\n3. Cool down: Stretching 
exercises for 5 minutes\n\nDays 6 & 7: Rest\n\nRepeat this weekly plan for 6 weeks. Ensure proper form and 
technique to avoid injuries. Gradually increase the weights as you progress through the weeks. Don\'t forget to 
maintain a balanced diet and stay hydrated to support your training and achieve your goals.\n===FORMAT 
INSTRUCTIONS: The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an 
example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": 
"array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a 
well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not 
well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"weeks": {"title": "Weeks", "description": 
"Organized series of weekly workout plans, creating a comprehensive and structured fitness program", "type": 
"array", "items": {"$ref": "#/definitions/Week"}}, "notes": {"title": "Notes", "description": "Supplementary 
information, tips or instructions to support and enhance the overall workout plan", "type": "array", "items": 
{"$ref": "#/definitions/Note"}}}, "required": ["weeks", "notes"], "definitions": {"Duration": {"title": "Duration",
"type": "object", "properties": {"value": {"title": "Value", "description": "Numerical value representing the 
duration of the exercise", "type": "integer"}, "unit": {"title": "Unit", "description": "Unit of time for the 
duration (e.g., seconds, minutes)", "type": "string"}}, "required": ["value", "unit"]}, "Exercise": {"title": 
"Exercise", "type": "object", "properties": {"name": {"title": "Name", "description": "Specific exercise to perform
in the workout plan", "type": "string"}, "sets": {"title": "Sets", "description": "Total number of sets of the 
exercise to complete", "type": "integer"}, "reps": {"title": "Reps", "description": "Total number of repetitions of
the exercise in each set", "type": "integer"}, "duration": {"title": "Duration", "description": "Duration of the 
exercise, if applicable", "allOf": [{"$ref": "#/definitions/Duration"}]}}, "required": ["name"]}, "Day": {"title": 
"Day", "type": "object", "properties": {"day_number": {"title": "Day Numb